# 🐾 Guess the Animal Game - AI Education Project

**Description:** Interactive game where AI gives clues and you guess the animal!

**Curriculum Phase:** C-D (Kelas 5-6 SD, Kelas 7-9 SMP)

**Difficulty:** Beginner to Intermediate

In this game, the AI will think of an animal and give you clues. Can you guess it before you run out of points? 🎮

## 📚 STEP 1: Install Dependencies

First, we need to install the required libraries

In [1]:
%pip install -q requests gradio

## 📚 STEP 2: Setup

Import the libraries and set up the backend URL

In [18]:
import base64
import requests
import gradio as gr
import json

# Obfuscated credentials
_BACKEND = base64.b64decode("aHR0cHM6Ly9sbG0tcHJveHktMzY0ODg2Mjk4ODY2LnVzLWNlbnRyYWwxLnJ1bi5hcHA=").decode()
_KEY = base64.b64decode("a2thLXNlY3JldC0yMDI1LXg3Sm05UHEyV241UnQ4THYzS3M2SGc0WXo=").decode()

## 📚 STEP 3: Game Logic Functions

These functions handle the game mechanics and communicate with the AI

In [19]:
# Function to call the AI backend and get response
def call_ai(prompt, endpoint="elementary"):
    """Call AI backend and get response"""
    try:
        response = requests.post(
            f"{_BACKEND}/chat/{endpoint}",
            headers={"X-API-Key": _KEY},
            json={"prompt": prompt},
            timeout=30
        )
        response.raise_for_status()
        return response.json()["response"]
    except Exception as e:
        return f"Error: {str(e)}"

# Function to start a new game
def start_new_game(difficulty):
    """Start a new game by asking AI to choose an animal"""
    endpoint = "elementary" if difficulty == "Mudah (SD)" else "middle"
    
    prompt = """Pilih satu hewan secara acak. Berikan 3 petunjuk singkat tentang hewan ini.
    
Format yang harus kamu gunakan:
HEWAN: [nama hewan]
1. [petunjuk pertama]
2. [petunjuk kedua]
3. [petunjuk ketiga]

Contoh:
HEWAN: Gajah
1. Hewan ini sangat besar dan berat
2. Memiliki belalai yang panjang
3. Telinganya lebar seperti kipas

Sekarang pilih hewan yang berbeda!"""
    
    ai_response = call_ai(prompt, endpoint)
    
    # Parse the response to extract animal name and clues
    lines = ai_response.strip().split('\n')
    animal = ""
    clues = []
    
    for line in lines:
        if line.startswith("HEWAN:"):
            animal = line.replace("HEWAN:", "").strip()
        elif line.strip() and (line[0].isdigit() or line.startswith("-")):
            clue = line.split(".", 1)[-1].strip() if "." in line else line.strip("- ")
            if clue:
                clues.append(clue)
    
    return {
        "animal": animal.lower(),
        "all_clues": clues,
        "shown_clues": 3,
        "points": 100,
        "game_over": False,
        "difficulty": difficulty,
        "endpoint": endpoint
    }

# Function to format the game state for display
def format_game_display(game_state):
    """Format the current game state for display"""
    if game_state["game_over"]:
        return game_state.get("final_message", "")
    
    display = f"""🎮 **TEBAK HEWAN!**\n
🔍 **Petunjuk:**\n"""
    
    num_clues = min(game_state['shown_clues'], len(game_state['all_clues']))
    for i in range(num_clues):
        display += f"{i+1}. {game_state['all_clues'][i]}\n"
    
    display += "\n💡 **Apa yang ingin kamu lakukan?**\n"
    display += "- Ketik tebakanmu di kotak di bawah, ATAU\n"
    display += "- Klik 'Minta Petunjuk Tambahan' (-10 poin)\n"
    
    return display


def format_points_display(points, game_over=False):
    """Format the points display"""
    if game_over:
        return f"""<div class="points-display">🏁 POIN AKHIR: {points}</div>"""
    return f"""<div class="points-display">💰 POIN: {points}</div>"""

# Function to check the guess
def check_guess(guess, game_state):
    """Check if the guess is correct"""
    if game_state["game_over"]:
        return (game_state, format_game_display(game_state), "", 
                format_points_display(game_state["points"], True),
                gr.update(interactive=False), gr.update(interactive=False))
    
    guess_clean = guess.lower().strip()
    animal_clean = game_state["animal"].lower().strip()
    
    # Check if guess is correct
    if guess_clean == animal_clean or guess_clean in animal_clean or animal_clean in guess_clean:
        game_state["game_over"] = True
        game_state["final_message"] = f"""🎉 **SELAMAT! KAMU MENANG!** 🎉\n
✅ Jawabannya memang **{game_state['animal'].upper()}**!\n
🏆 **Poin akhir:** {game_state['points']} poin\n
Kerja bagus! Kamu berhasil menebak dengan {game_state['shown_clues']} petunjuk! 🌟\n
Klik 'Mulai Game Baru' untuk bermain lagi!"""
    else:
        game_state["game_over"] = True
        game_state["final_message"] = f"""😢 **MAAF, SALAH!** \n
❌ Tebakanmu: **{guess}**\n
✅ Jawaban yang benar: **{game_state['animal'].upper()}**\n
Jangan menyerah! Coba lagi dan kamu pasti bisa! 💪\n
Klik 'Mulai Game Baru' untuk mencoba lagi!"""
    
    return (game_state, format_game_display(game_state), "", 
            format_points_display(game_state["points"], True),
            gr.update(interactive=False), gr.update(interactive=False))

# Function to request additional clues
def request_more_clues(game_state):
    """Request an additional clue from AI"""
    if game_state["game_over"]:
        return game_state, format_game_display(game_state), format_points_display(game_state["points"], True)
    
    # Deduct points
    game_state["points"] = max(0, game_state["points"] - 10)
    
    # Check if we need to generate a new clue
    if game_state["shown_clues"] >= len(game_state["all_clues"]):
        # Generate a new clue from AI
        prompt = f"""Hewan yang sedang ditebak adalah {game_state['animal']}.
        
Petunjuk yang sudah diberikan:
{chr(10).join([f'{i+1}. {clue}' for i, clue in enumerate(game_state['all_clues'])])}

Berikan 1 petunjuk tambahan yang berbeda dan menarik tentang hewan ini. 
Jawab HANYA dengan petunjuknya saja, jangan tambahkan nomor atau kata-kata lain."""
        
        new_clue = call_ai(prompt, game_state["endpoint"])
        game_state["all_clues"].append(new_clue.strip())
    
    game_state["shown_clues"] += 1
    
    return game_state, format_game_display(game_state), format_points_display(game_state["points"])


## 📚 STEP 4: Create Gradio Interface

Build an interactive and colorful interface for the game

In [20]:
# Custom CSS for a more engaging look
custom_css = """
.game-title {
    text-align: center;
    color: #2196F3;
    font-size: 2em;
    margin-bottom: 20px;
}
.instruction-box {
    padding: 15px;
    border-radius: 10px;
    border-left: 5px solid #FF9800;
}
.points-display {
    text-align: center;
    font-size: 3em;
    font-weight: bold;
    padding: 30px;
    border-radius: 10px;
    background: #667eea;
    color: white;
}
"""

with gr.Blocks(css=custom_css, theme=gr.themes.Soft()) as game_interface:
    gr.Markdown(
        """
        # 🐾 TEBAK HEWAN! 🎯
        
        Permainan seru di mana AI akan memberikan petunjuk dan kamu harus menebak hewannya!
        """,
        elem_classes="game-title"
    )
    
    # Game state (hidden)
    game_state = gr.State(value={
        "animal": "",
        "all_clues": [],
        "shown_clues": 0,
        "points": 100,
        "game_over": True,
        "difficulty": "Mudah (SD)",
        "endpoint": "elementary"
    })
    
    # Large points display
    points_display = gr.HTML(
        """<div class="points-display">💰 POIN: --</div>"""
    )
    
    with gr.Row():
        with gr.Column(scale=3):
            # Difficulty selector
            difficulty_selector = gr.Radio(
                choices=["Mudah (SD)", "Menengah (SMP)"],
                value="Mudah (SD)",
                label="🎚️ Tingkat Kesulitan"
            )
            
            # Start button
            start_btn = gr.Button("🚀 Mulai Game Baru", variant="primary", size="lg")
            
            # Game display
            game_display = gr.Markdown(
                """### 👋 Selamat Datang!
                
Klik tombol 'Mulai Game Baru' untuk memulai petualanganmu! 🎮"""
            )
            
            # Guess input
            guess_input = gr.Textbox(
                label="✍️ Tebakanmu",
                placeholder="Ketik nama hewan di sini...",
                interactive=False
            )
            
            with gr.Row():
                submit_btn = gr.Button("✅ Tebak!", variant="primary", interactive=False)
                more_clues_btn = gr.Button("💡 Minta Petunjuk Tambahan (-10 poin)", interactive=False)
        
        with gr.Column(scale=2):
            with gr.Accordion("📖 Cara Bermain", open=True):
                gr.Markdown(
                    """
                    1. 🎮 Pilih tingkat kesulitan dan klik 'Mulai Game Baru'
                    2. 🔍 AI akan memberikan 3 petunjuk tentang sebuah hewan
                    3. 🤔 Pikirkan dan ketik tebakanmu
                    4. 💡 Butuh bantuan? Klik 'Minta Petunjuk Tambahan' (-10 poin)
                    5. 🏆 Tebak dengan cepat untuk mendapatkan poin maksimal!
                    
                    **Poin awal:** 100 poin  
                    **Setiap petunjuk tambahan:** -10 poin
                    """
                )
    
    # Example animals for inspiration
    gr.Markdown(
        """
        ---
        ### 🦁 Contoh Hewan yang Mungkin Muncul:
        Singa, Gajah, Jerapah, Penguin, Panda, Harimau, Beruang, Kura-kura, Lumba-lumba, Burung Hantu, dan banyak lagi!
        """
    )
    
    # Event handlers
    def on_start_game(difficulty):
        new_state = start_new_game(difficulty)
        display = format_game_display(new_state)
        points = format_points_display(new_state["points"])
        return (
            new_state,
            display,
            points,
            gr.update(interactive=True, value=""),
            gr.update(interactive=True),
            gr.update(interactive=True)
        )
    
    start_btn.click(
        fn=on_start_game,
        inputs=[difficulty_selector],
        outputs=[game_state, game_display, points_display, guess_input, submit_btn, more_clues_btn]
    )
    
    submit_btn.click(
        fn=check_guess,
        inputs=[guess_input, game_state],
        outputs=[game_state, game_display, guess_input, points_display, submit_btn, more_clues_btn]
    )
    
    guess_input.submit(
        fn=check_guess,
        inputs=[guess_input, game_state],
        outputs=[game_state, game_display, guess_input, points_display, submit_btn, more_clues_btn]
    )
    
    more_clues_btn.click(
        fn=request_more_clues,
        inputs=[game_state],
        outputs=[game_state, game_display, points_display]
    )


## 📚 STEP 5: Launch the Game!

Run this cell to start playing! 🎮

In [21]:
game_interface.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b8e1c7e2cc9f6f24c3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://b8e1c7e2cc9f6f24c3.gradio.live


### 💡 LEARNING SECTION

#### What's happening in this game?

1. **AI-Powered Game Master**: The AI acts as a game master, choosing animals and creating clues dynamically.

2. **State Management**: We use Gradio's `State` component to keep track of:
   - The secret animal
   - All clues (including newly generated ones)
   - Current points
   - Whether the game is over

3. **Dynamic Content Generation**: Each clue is generated by AI on-demand, making every game unique!

4. **String Matching**: We use Python's string operations to check if your guess matches the animal name.

5. **Educational Levels**: Different difficulty levels use different AI endpoints:
   - **Mudah (SD)**: Uses simpler language for elementary students
   - **Menengah (SMP)**: Uses more detailed descriptions for middle school

#### Try these challenges:

1. 🎯 Can you win with 100 points (guess after just 3 clues)?
2. 🌟 Try both difficulty levels - notice the difference in clues?
3. 🤔 Can you guess animals from different categories (mammals, birds, sea creatures)?

#### Extension Ideas:

- Add a leaderboard to track high scores
- Include hints categories (habitat, diet, size)
- Add sound effects or images when you win
- Create themed games (sea animals only, endangered species, etc.)

#### Questions to think about:

- How does the AI generate relevant clues without repeating?
- What makes a good clue - too easy or too hard?
- Could you create a similar game for other topics (countries, famous people, etc.)?

Have fun playing! 🎉
